<a href="https://colab.research.google.com/github/matthew110395/12004210_DataAnalytics/blob/main/12004210_DAOTW_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction
This assignment builds on the New York taxi problem identified in assignment 1, where the City of New York is looking for a way to accurately predict the number of collisions on a particular day of the week. Within this document two different types of machine learning models will be utilised to predict the number of collisions. The linear relationships identified in assignment 1 will be used to create a linear regression model. A Deep Learning Neural Network (DNN) will be used to predict the number of collisions where the relationship is not linear. 

#Imports
To prepare data for machine learning the pandas package has been used. The numpy package has been used to aid with mathematical functions.

As within part 1 of this assignment the data file containing location data exceeds the size limit for hosting within github. To overcome this the file was zipped. In order to extract the data from the zip file the zipfile package has been used.

Within this document, Tensorflow has been used for machine learning, with both a linear regression model and a Deep Neural Network model. Tensorflow version 1 is unsupported within Google Colab, therefore must be installed using the pip package manager.

Shutil has also been imported to allow for easy file management, in particular the removal of saved models.

In [104]:
import pandas as pd
import numpy as np
import zipfile


!pip install tensorflow==1.15.2
import tensorflow as tf
# needed for high-level file management
import shutil  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Linear Regressor
Throughout assignment 1 a number of linear relationships were uncovered within the dataset. These relationsips form the basis of the linear regression models below.



##Precipition

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/prcp_clean.csv', index_col=0, )
print(df[:6])

   day  year  mo  da collision_date  NUM_COLLISIONS  temp  dewp     slp  \
1    5  2020   1  24     2020-01-24             524  37.3  33.7  1028.5   
2    2  2021   1  12     2021-01-12             278  37.0  29.1  1019.0   
3    5  2021   1  22     2021-01-22             254  36.5  28.4  1003.1   
4    3  2021   1  27     2021-01-27             262  34.6  33.8  1012.8   
5    2  2021   1  26     2021-01-26             263  31.9  23.4  1016.9   
6    1  2022   1  24     2022-01-24             237  34.5  23.8  1010.6   

   visib  ...   max   min  prcp   sndp  fog  rain_drizzle  snow_ice_pellets  \
1    6.5  ...  46.0  19.9  0.00  999.9    1             0                 0   
2   10.0  ...  44.1  21.0  0.00  999.9    0             0                 0   
3   10.0  ...  44.1  19.9  0.00  999.9    0             0                 0   
4    8.0  ...  41.0  28.9  0.25  999.9    1             1                 0   
5    9.0  ...  37.9  21.0  0.00  999.9    1             0                 0   


In [3]:
df_prcp = df.drop(columns=['collision_date', 'temp', 'dewp', 'slp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud'])
df_prcp = df_prcp.loc[df_prcp["year"] != 2012]
df_prcp = df_prcp.loc[df_prcp["year"] < 2020]
cols = df_prcp['NUM_COLLISIONS']
df_prcp = df_prcp.drop(columns=['NUM_COLLISIONS'])
df_prcp.insert(loc=9, column='NUM_COLLISIONS', value=cols)
print(df_prcp[:6])
df_prcp.describe()

    day  year  mo  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail  \
49    4  2016   1  28  0.09    0             0                 0     0   
51    5  2014   1  17  0.00    1             0                 0     0   
54    1  2016   1  25  0.02    0             0                 0     0   
55    5  2016   1  29  0.00    0             0                 0     0   
58    5  2017   1  20  0.00    0             0                 0     0   
59    7  2013   1  13  0.01    1             0                 0     0   

    NUM_COLLISIONS  
49             681  
51             589  
54             658  
55             645  
58             605  
59             373  


,day,year,mo,da,prcp,fog,rain_drizzle,snow_ice_pellets,hail,NUM_COLLISIONS
count,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000
mean,3.998425,2015.989366,6.518708,15.745569,0.122588,0.253249,0.375345,0.085467,0.000394,599.135093
std,2.003542,1.996126,3.455211,8.803199,0.329143,0.434958,0.484307,0.279630,0.019846,100.299164
min,1.000000,2013.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2.000000,2014.000000,4.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000
50%,4.000000,2016.000000,7.000000,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000
75%,6.000000,2018.000000,10.000000,23.000000,0.060000,1.000000,1.000000,0.000000,0.000000,665.000000
max,7.000000,2019.000000,12.000000,31.000000,3.760000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [4]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_prcp.iloc[np.random.permutation(len(df_prcp))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]


# print out the first 6 rows of predictors.
print(predictors[:6])
print(shuffle[:6])

      day  year  mo  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail
213     3  2014   1   1  0.00    0             0                 0     0
2023    6  2015   7  11  0.00    0             0                 0     0
1473    5  2017   5  26  0.10    1             1                 0     0
2996    1  2018  10  29  0.00    0             1                 0     0
1151    4  2015   4   9  0.18    0             1                 0     0
240     2  2015   1  13  0.82    0             1                 1     0
      day  year  mo  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail  \
213     3  2014   1   1  0.00    0             0                 0     0   
2023    6  2015   7  11  0.00    0             0                 0     0   
1473    5  2017   5  26  0.10    1             1                 0     0   
2996    1  2018  10  29  0.00    0             1                 0     0   
1151    4  2015   4   9  0.18    0             1                 0     0   
240     2  2015   1  13  0.82    

In [5]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

213     399
2023    597
1473    776
2996    696
1151    452
240     613
Name: NUM_COLLISIONS, dtype: int64


In [6]:
SCALE_COLLISIONS=1161

In [7]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
print(trainsize)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize
print(testsize)
# Define the number of input values (predictors)
nppredictors = 9
# Define the number of output values (targets)
noutputs = 1

2031
508


In [8]:
# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model_prcp', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_prcp', optimizer=tf.train.AdamOptimizer(learning_rate=0.001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))
print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)
preds = estimator.predict(x=predictors[trainsize:].values)
#print(preds)
predslistscale = preds['scores']*SCALE_COLLISIONS

pred = format(str(predslistscale))
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please specify feature columns explicitly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please feed input to tf.data to support dask.
Instructions for updating:
Please access pandas data directly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please convert numpy dtypes explicitly.
Instructions for updating:
Please specify feature columns explicitly.
Instructions for updating:
Please switch to tf.contrib.estimator.*_head.
Instructions for updating:
Please replace uses of any Estimator from tf.contri

starting to train


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/linear_regression_trained_model_prcp/model.ckpt.
INFO:tensorflow:loss = 0.2718349, step = 1
INFO:tensorflow:global_step/sec: 1274.24
INFO:tensorflow:loss = 0.0070736543, step = 101 (0.083 sec)
INFO:tensorflow:global_step/sec: 1480.03
INFO:tensorflow:loss = 0.0050733453, step = 201 (0.065 sec)
INFO:tensorflow:global_step/sec: 1310.89
INFO:tensorflow:loss = 0.007885993, step = 301 (0.077 sec)
INFO:tensorflow:global_step/sec: 1457.39
INFO:tensorflow:loss = 0.006817926, step = 401 (0.071 sec)
INFO:tensorflow:global_step/sec: 1369.62
INFO:tensorflow:loss = 0.0076487446, step = 501 (0.070 sec)
INFO:tensorflow:global_step/sec: 1308.28
INFO:tensorflow:loss = 0.0065248143, step = 601 (0.076 sec)
INFO:tensorflow:global_step/sec: 1264.2

LinearRegression has RMSE of 120.27965286688338
Just using average = 598.9635647464303 has RMSE of 99.45388340325029


In [9]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_prcp', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff06543cf50>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_prcp', '_session_creation_timeout_secs': 7200}


508
508


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_prcp/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.46280783 0.39800078 0.4636122  0.42282948 0.46114007 0.43913487
 0.449243   0.47614846 0.4382128  0.45474368 0.44615972 0.41813916
 0.4794206  0.47818336 0.49988747 0.45169005 0.4742704  0.42404553
 0.46931753 0.49836254 0.43093872 0.51345295 0.46592873 0.4660528
 0.486162   0.4725375  0.49260208 0.5088301  0.4666033  0.4598983
 0.4463999  0.50934154 0.4714387  0.44519517 0.42109233 0.47672996
 0.43556514 0.4130507  0.46865654 0.43353376 0.45110175 0.4701876
 0.4564987  0.41614628 0.41391096 0.42304745 0.46023485 0.4267523
 0.42839003 0.43011484 0.43222958 0.49868333 0.49761024 0.48804387
 0.44515806 0.5009714  0.42252412 0.3916985  0.4185493  0.4585322
 0.47623903 0.4829306  0.4543146  0.47516623 0.46681905 0.4869174
 0.42103022 0.43544436 0.43491256 0.50192577 0.4223419  0.44819778
 0.42044258 0.45645896 0.47989038 0.46869466 0.4944869  0.47781485
 0.4751654  0.4240815  0.45045096 0.38560256 0.44755152 0.40396866
 0.38276157 0.5017927  0.44771236 0.47452453 0.45047826 0.48305547
 

##Dew Point (dewp)


In [10]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/dewp_clean.csv', index_col=0, )
print(df[:6])

   day  year  mo  da collision_date  NUM_COLLISIONS  temp  dewp     slp  \
1    5  2020   1  24     2020-01-24             524  37.3  33.7  1028.5   
2    2  2021   1  12     2021-01-12             278  37.0  29.1  1019.0   
3    5  2021   1  22     2021-01-22             254  36.5  28.4  1003.1   
4    3  2021   1  27     2021-01-27             262  34.6  33.8  1012.8   
5    2  2021   1  26     2021-01-26             263  31.9  23.4  1016.9   
6    1  2022   1  24     2022-01-24             237  34.5  23.8  1010.6   

   visib  ...   max   min  prcp   sndp  fog  rain_drizzle  snow_ice_pellets  \
1    6.5  ...  46.0  19.9  0.00  999.9    1             0                 0   
2   10.0  ...  44.1  21.0  0.00  999.9    0             0                 0   
3   10.0  ...  44.1  19.9  0.00  999.9    0             0                 0   
4    8.0  ...  41.0  28.9  0.25  999.9    1             1                 0   
5    9.0  ...  37.9  21.0  0.00  999.9    1             0                 0   


In [11]:
df_dewp = df.drop(columns=['collision_date', 'temp', 'prcp', 'slp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_dewp = df_dewp.loc[df_dewp["year"] != 2012]
df_dewp = df_dewp.loc[df_dewp["year"] < 2020]
cols = df_dewp['NUM_COLLISIONS']
df_dewp = df_dewp.drop(columns=['NUM_COLLISIONS'])
df_dewp.insert(loc=5, column='NUM_COLLISIONS', value=cols)
print(df_dewp[:6])
df_dewp.describe()

    day  year  mo  da  dewp  NUM_COLLISIONS
49    4  2016   1  28  24.4             681
51    5  2014   1  17  35.8             589
54    1  2016   1  25  21.2             658
55    5  2016   1  29  36.8             645
58    5  2017   1  20  32.5             605
59    7  2013   1  13  44.9             373


,day,year,mo,da,dewp,NUM_COLLISIONS
count,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000
mean,3.998434,2015.999217,6.524070,15.723679,44.163170,599.109980
std,2.000391,2.000000,3.449676,8.801271,16.995303,100.277185
min,1.000000,2013.000000,1.000000,1.000000,-6.700000,188.000000
25%,2.000000,2014.000000,4.000000,8.000000,32.150000,531.000000
50%,4.000000,2016.000000,7.000000,16.000000,45.300000,602.000000
75%,6.000000,2018.000000,10.000000,23.000000,58.500000,665.000000
max,7.000000,2019.000000,12.000000,31.000000,74.100000,1161.000000


In [12]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_dewp.iloc[np.random.permutation(len(df_dewp))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]


# print out the first 6 rows of predictors.
print(predictors[:6])
print(shuffle[:6])

      day  year  mo  da  dewp
453     3  2018   2  28  38.2
983     6  2014   4  12  40.8
2120    4  2016   7  14  65.9
744     1  2016   3  28  41.4
2705    5  2014   9  19  42.7
3489    7  2019  12  22  34.9
      day  year  mo  da  dewp  NUM_COLLISIONS
453     3  2018   2  28  38.2             654
983     6  2014   4  12  40.8             578
2120    4  2016   7  14  65.9             741
744     1  2016   3  28  41.4             658
2705    5  2014   9  19  42.7             648
3489    7  2019  12  22  34.9             510


In [13]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

453     654
983     578
2120    741
744     658
2705    648
3489    510
Name: NUM_COLLISIONS, dtype: int64


In [14]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
print(trainsize)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize
print(testsize)
# Define the number of input values (predictors)
nppredictors = 5
# Define the number of output values (targets)
noutputs = 1

2044
511


In [15]:
# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model_dewp', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_dewp', optimizer=tf.train.AdamOptimizer(learning_rate=0.0001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))
print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)
preds = estimator.predict(x=predictors[trainsize:].values)
#print(preds)
predslistscale = preds['scores']*SCALE_COLLISIONS

pred = format(str(predslistscale))
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff05e6b1a90>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_dewp', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/linear_regression_trained_model_dewp/model.ckpt.
INFO:tensorflow:loss = 0.28520566, step = 1
INFO:tensorflow:global_step/sec: 925.282
INFO:tensorflow:loss = 0.008250323, step = 101 (0.112 sec)
INFO:tensorflow:global_step/sec: 1472.21
INFO:tensorflow:loss = 0.0055447062, step = 201 (0.065 sec)
INFO:tensorflow:global_step/sec: 1456.22
INFO:tensorflow:loss = 0.0073354435, step = 301 (0.071 sec)
INFO:tensorflow:global_step/sec: 1422.52
INFO:tensorflow:loss = 0.0069964584, step = 401 (0.071 sec)
INFO:tensorflow:global_step/sec: 1376.37
INFO:tensorflow:loss = 0.006039256, step = 501 (0.073 sec)
INFO:tensorflow:global_step/sec: 1275.28
INFO:tensorflow:loss = 0.005632888, step = 601 (0.078 sec)
INFO:tensorflow:global_step/sec: 1446.33
INFO:tensorflow:loss = 0.006950683, step = 701 (

LinearRegression has RMSE of 97.94437967770767
Just using average = 599.2808219178082 has RMSE of 102.93516150827149


In [16]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_dewp', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff0633140d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_dewp', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_dew

511
511


INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.50059634 0.47600403 0.5100045  0.538167   0.5501877  0.47055468
 0.5253374  0.49427247 0.4663128  0.53107166 0.51078576 0.5553396
 0.5104437  0.5738878  0.519115   0.47461873 0.44475165 0.52343017
 0.5296347  0.4917729  0.48960534 0.45493123 0.4693856  0.48472452
 0.5681601  0.48693624 0.47232756 0.4571172  0.56385714 0.54767275
 0.5015362  0.5185676  0.50891745 0.5047301  0.49179333 0.5014119
 0.53723615 0.5529622  0.498054   0.53834313 0.4897869  0.4926679
 0.56658465 0.48149166 0.4754755  0.51804906 0.55515873 0.5401877
 0.4655891  0.4660888  0.4817309  0.46763402 0.53577554 0.51789695
 0.4939717  0.52303934 0.55928564 0.5311088  0.51918095 0.47887585
 0.5129394  0.45448965 0.5673249  0.44762564 0.54415876 0.5492752
 0.49934    0.48102468 0.52760416 0.46029854 0.52262765 0.49600267
 0.5136431  0.5013631  0.48088166 0.55106974 0.4518191  0.49173287
 0.53584874 0.4596553  0.52352434 0.5009683  0.48000833 0.53057534
 0.48602706 0.49181232 0.49291882 0.571389   0.4901338  0.5111236
 

##Sea Level Pressure (SLP)


In [17]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/slp_clean.csv', index_col=0, )
print(df[:6])

   day  year  mo  da collision_date  NUM_COLLISIONS  temp  dewp     slp  \
1    5  2020   1  24     2020-01-24             524  37.3  33.7  1028.5   
2    2  2021   1  12     2021-01-12             278  37.0  29.1  1019.0   
3    5  2021   1  22     2021-01-22             254  36.5  28.4  1003.1   
4    3  2021   1  27     2021-01-27             262  34.6  33.8  1012.8   
5    2  2021   1  26     2021-01-26             263  31.9  23.4  1016.9   
6    1  2022   1  24     2022-01-24             237  34.5  23.8  1010.6   

   visib  ...   max   min  prcp   sndp  fog  rain_drizzle  snow_ice_pellets  \
1    6.5  ...  46.0  19.9  0.00  999.9    1             0                 0   
2   10.0  ...  44.1  21.0  0.00  999.9    0             0                 0   
3   10.0  ...  44.1  19.9  0.00  999.9    0             0                 0   
4    8.0  ...  41.0  28.9  0.25  999.9    1             1                 0   
5    9.0  ...  37.9  21.0  0.00  999.9    1             0                 0   


In [18]:
df_slp = df.drop(columns=['collision_date', 'temp', 'prcp', 'dewp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_slp = df_slp.loc[df_slp["year"] != 2012]
df_slp = df_slp.loc[df_slp["year"] < 2020]
cols = df_slp['NUM_COLLISIONS']
df_slp = df_slp.drop(columns=['NUM_COLLISIONS'])
df_slp.insert(loc=5, column='NUM_COLLISIONS', value=cols)
print(df_slp[:6])
df_slp.describe()

    day  year  mo  da     slp  NUM_COLLISIONS
49    4  2016   1  28  1016.1             681
51    5  2014   1  17  1014.8             589
54    1  2016   1  25  1021.4             658
55    5  2016   1  29   999.4             645
58    5  2017   1  20  1015.5             605
59    7  2013   1  13  1020.7             373


,day,year,mo,da,slp,NUM_COLLISIONS
count,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000
mean,3.999217,2016.000391,6.522114,15.719765,1016.777221,599.147162
std,2.000783,2.000294,3.447986,8.796698,7.628429,100.268048
min,1.000000,2013.000000,1.000000,1.000000,989.500000,188.000000
25%,2.000000,2014.000000,4.000000,8.000000,1012.200000,531.000000
50%,4.000000,2016.000000,7.000000,16.000000,1016.700000,602.000000
75%,6.000000,2018.000000,10.000000,23.000000,1021.700000,665.000000
max,7.000000,2019.000000,12.000000,31.000000,1044.200000,1161.000000


In [19]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_slp.iloc[np.random.permutation(len(df_slp))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]


# print out the first 6 rows of predictors.
print(predictors[:6])
print(shuffle[:6])

      day  year  mo  da     slp
1758    4  2019   6   6  1007.5
831     3  2016   3  16  1007.8
174     1  2017   1   9  1035.5
2858    1  2014  10   6  1018.6
2385    4  2016   8  11  1018.5
576     3  2015   2  11  1014.0
      day  year  mo  da     slp  NUM_COLLISIONS
1758    4  2019   6   6  1007.5             744
831     3  2016   3  16  1007.8             629
174     1  2017   1   9  1035.5             669
2858    1  2014  10   6  1018.6             639
2385    4  2016   8  11  1018.5             672
576     3  2015   2  11  1014.0             610


In [20]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

1758    744
831     629
174     669
2858    639
2385    672
576     610
Name: NUM_COLLISIONS, dtype: int64


In [21]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
print(trainsize)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize
print(testsize)
# Define the number of input values (predictors)
nppredictors = 5
# Define the number of output values (targets)
noutputs = 1

2044
511


In [22]:
# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model_slp', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_slp', optimizer=tf.train.AdamOptimizer(learning_rate=0.0001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))
print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)
preds = estimator.predict(x=predictors[trainsize:].values)
#print(preds)
predslistscale = preds['scores']*SCALE_COLLISIONS

pred = format(str(predslistscale))
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff065487b10>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_slp', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/linear_regression_trained_model_slp/model.ckpt.
INFO:tensorflow:loss = 0.28725958, step = 1
INFO:tensorflow:global_step/sec: 493.866
INFO:tensorflow:loss = 0.007920365, step = 101 (0.205 sec)
INFO:tensorflow:global_step/sec: 684.571
INFO:tensorflow:loss = 0.008888739, step = 201 (0.144 sec)
INFO:tensorflow:global_step/sec: 875.031
INFO:tensorflow:loss = 0.009648033, step = 301 (0.121 sec)
INFO:tensorflow:global_step/sec: 614.28
INFO:tensorflow:loss = 0.007889533, step = 401 (0.158 sec)
INFO:tensorflow:global_step/sec: 674.63
INFO:tensorflow:loss = 0.006255895, step = 501 (0.149 sec)
INFO:tensorflow:global_step/sec: 798.949
INFO:tensorflow:loss = 0.006308317, step = 601 (0.129 sec)
INFO:tensorflow:global_step/sec: 424.679
INFO:tensorflow:loss = 0.0069407383, step = 701 (0.229

LinearRegression has RMSE of 95.13112435157943
Just using average = 598.6340508806262 has RMSE of 101.05888584014674


In [23]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_slp', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff0654e6b50>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_slp', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_slp/

511
511
[0.52682537 0.49981183 0.50644207 0.44715485 0.48286563 0.53737247
 0.5141071  0.5589524  0.52432066 0.54054755 0.525875   0.49438748
 0.5254183  0.51265687 0.51494324 0.4689978  0.53749907 0.49513215
 0.5295475  0.45016575 0.53822476 0.4784605  0.51097596 0.49904215
 0.52671385 0.50889647 0.48168465 0.5588118  0.51342297 0.5197076
 0.51154536 0.4758883  0.45808664 0.50623    0.5348296  0.52293646
 0.4867742  0.4672252  0.49655572 0.49356687 0.5587935  0.5211494
 0.5678288  0.48533905 0.5426654  0.5393282  0.51707697 0.4684079
 0.52812093 0.5529818  0.5123113  0.47001424 0.47784555 0.53001744
 0.48605138 0.48843557 0.5167751  0.5278232  0.52132607 0.50046384
 0.5042566  0.51408064 0.4951198  0.5463461  0.4888631  0.51103014
 0.53763545 0.54957104 0.53674024 0.5571053  0.5411252  0.48911384
 0.48324132 0.51716036 0.5345853  0.51393646 0.48754728 0.55031455
 0.49590802 0.46609488 0.5654458  0.5165404  0.4954292  0.54154015
 0.5510669  0.48657876 0.52368623 0.51143825 0.56101054 0

##Gust

In [24]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/gust_clean.csv', index_col=0, )
print(df[:6])

    day  year  mo  da collision_date  NUM_COLLISIONS  temp  dewp     slp  \
3     5  2021   1  22     2021-01-22             254  36.5  28.4  1003.1   
11    3  2020   1  15     2020-01-15             508  43.9  38.3  1019.4   
12    5  2021   1   1     2021-01-01             257  39.6  29.3  1029.3   
14    2  2022   1  25     2022-01-25             235  41.6  31.8  1013.2   
18    7  2021   1   3     2021-01-03             186  41.1  32.3  1018.0   
19    4  2020   1   2     2020-01-02             413  39.6  28.9  1011.8   

    visib  ...   max   min  prcp   sndp  fog  rain_drizzle  snow_ice_pellets  \
3    10.0  ...  44.1  19.9  0.00  999.9    0             0                 0   
11    8.2  ...  51.1  35.1  0.02  999.9    1             0                 0   
12   10.0  ...  54.0  33.1  0.13  999.9    0             0                 0   
14   10.0  ...  48.9  30.0  0.00  999.9    0             0                 0   
18   10.0  ...  53.1  39.0  0.00  999.9    0             0         

In [25]:
df_gust = df.drop(columns=['collision_date', 'temp', 'prcp', 'slp','visib','max','min','sndp','wdsp','mxpsd','dewp','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_gust = df_gust.loc[df_gust["year"] != 2012]
df_gust = df_gust.loc[df_gust["year"] < 2020]
cols = df_gust['NUM_COLLISIONS']
df_gust = df_gust.drop(columns=['NUM_COLLISIONS'])
df_gust.insert(loc=5, column='NUM_COLLISIONS', value=cols)
print(df_gust[:6])
df_gust.describe()

    day  year  mo  da  gust  NUM_COLLISIONS
74    7  2016   1  17  18.1             451
76    4  2014   1   9  20.0             561
79    6  2019   1  19  21.0             479
80    7  2015   1  11  17.1             341
83    4  2015   1  29  20.0             519
85    7  2019   1  13  15.9             374


,day,year,mo,da,gust,NUM_COLLISIONS
count,1629.000000,1629.00000,1629.000000,1629.000000,1629.000000,1629.000000
mean,4.024555,2015.91283,6.278699,15.702885,27.511602,596.513198
std,1.989070,2.01341,3.747683,8.667634,7.366770,104.479660
min,1.000000,2013.00000,1.000000,1.000000,14.000000,188.000000
25%,2.000000,2014.00000,3.000000,8.000000,22.000000,526.000000
50%,4.000000,2016.00000,6.000000,16.000000,26.000000,597.000000
75%,6.000000,2018.00000,10.000000,23.000000,31.100000,663.000000
max,7.000000,2019.00000,12.000000,31.000000,71.100000,1161.000000


In [26]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_gust.iloc[np.random.permutation(len(df_gust))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]


# print out the first 6 rows of predictors.
print(predictors[:6])
print(shuffle[:6])

      day  year  mo  da  gust
1764    7  2019   6  16  24.1
3338    7  2016  11  20  36.9
2689    6  2016   9  24  24.1
2120    4  2016   7  14  22.0
3581    4  2013  12  19  27.0
3308    6  2019  11  30  29.9
      day  year  mo  da  gust  NUM_COLLISIONS
1764    7  2019   6  16  24.1             509
3338    7  2016  11  20  36.9             529
2689    6  2016   9  24  24.1             612
2120    4  2016   7  14  22.0             741
3581    4  2013  12  19  27.0             607
3308    6  2019  11  30  29.9             465


In [27]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

1764    509
3338    529
2689    612
2120    741
3581    607
3308    465
Name: NUM_COLLISIONS, dtype: int64


In [28]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
print(trainsize)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize
print(testsize)
# Define the number of input values (predictors)
nppredictors = 5
# Define the number of output values (targets)
noutputs = 1

1303
326


In [29]:
# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model_gust', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_gust', optimizer=tf.train.AdamOptimizer(learning_rate=0.00001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))
print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)
preds = estimator.predict(x=predictors[trainsize:].values)
#print(preds)
predslistscale = preds['scores']*SCALE_COLLISIONS

pred = format(str(predslistscale))
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff05e67ca90>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_gust', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Create CheckpointSaverHook.


starting to train


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/linear_regression_trained_model_gust/model.ckpt.
INFO:tensorflow:loss = 0.28196698, step = 1
INFO:tensorflow:global_step/sec: 1123.22
INFO:tensorflow:loss = 0.0086251795, step = 101 (0.093 sec)
INFO:tensorflow:global_step/sec: 1290.03
INFO:tensorflow:loss = 0.008044067, step = 201 (0.075 sec)
INFO:tensorflow:global_step/sec: 1294.89
INFO:tensorflow:loss = 0.00948257, step = 301 (0.082 sec)
INFO:tensorflow:global_step/sec: 1342.43
INFO:tensorflow:loss = 0.009223659, step = 401 (0.076 sec)
INFO:tensorflow:global_step/sec: 1414.6
INFO:tensorflow:loss = 0.009794172, step = 501 (0.067 sec)
INFO:tensorflow:global_step/sec: 1357.52
INFO:tensorflow:loss = 0.008056955, step = 601 (0.073 sec)
INFO:tensorflow:global_step/sec: 1411.74
INFO:tensorflow:loss = 0.006747408, step = 701 (0.070 sec)
INFO:tensorflow:global_step/sec: 1493

LinearRegression has RMSE of 94.98848753278486
Just using average = 599.4090560245587 has RMSE of 102.91164198363217


In [30]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_gust', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff063120090>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_gust', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_gus

326
326
[0.45284078 0.547738   0.47461343 0.51207757 0.46765155 0.49457788
 0.503318   0.48984322 0.5414449  0.47987384 0.5143559  0.57154804
 0.46861786 0.4493055  0.49599162 0.4902836  0.4754573  0.50574386
 0.44684383 0.53113925 0.52525324 0.48205996 0.553042   0.4711395
 0.4901308  0.51685643 0.4807198  0.49802354 0.45696    0.46566376
 0.5210303  0.512185   0.49370024 0.569932   0.4945628  0.48314148
 0.56605995 0.52818453 0.4448419  0.48563427 0.49849766 0.48250577
 0.46955365 0.5347459  0.51083946 0.46002743 0.514167   0.48035315
 0.5484611  0.5314928  0.4413822  0.5181895  0.5191861  0.4483724
 0.4379755  0.4734558  0.44937906 0.48535302 0.50331396 0.48243588
 0.46735764 0.4664756  0.47279543 0.47518295 0.51549006 0.5512336
 0.53743726 0.56948483 0.44908753 0.55321026 0.5306957  0.49990833
 0.4903923  0.43070406 0.48095098 0.47428897 0.5625574  0.50087976
 0.52210635 0.5327489  0.5211852  0.5579033  0.43109837 0.52885264
 0.5118199  0.51344097 0.51326454 0.48555812 0.52339035 0

##Maximum Sustained Wind Speed (Mxpsd)

In [31]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/mxpsd_clean.csv', index_col=0, )
print(df[:6])

   day  year  mo  da collision_date  NUM_COLLISIONS  temp  dewp     slp  \
1    5  2020   1  24     2020-01-24             524  37.3  33.7  1028.5   
2    2  2021   1  12     2021-01-12             278  37.0  29.1  1019.0   
3    5  2021   1  22     2021-01-22             254  36.5  28.4  1003.1   
4    3  2021   1  27     2021-01-27             262  34.6  33.8  1012.8   
5    2  2021   1  26     2021-01-26             263  31.9  23.4  1016.9   
6    1  2022   1  24     2022-01-24             237  34.5  23.8  1010.6   

   visib  ...   max   min  prcp   sndp  fog  rain_drizzle  snow_ice_pellets  \
1    6.5  ...  46.0  19.9  0.00  999.9    1             0                 0   
2   10.0  ...  44.1  21.0  0.00  999.9    0             0                 0   
3   10.0  ...  44.1  19.9  0.00  999.9    0             0                 0   
4    8.0  ...  41.0  28.9  0.25  999.9    1             1                 0   
5    9.0  ...  37.9  21.0  0.00  999.9    1             0                 0   


In [32]:
df_mxpsd = df.drop(columns=['collision_date', 'temp', 'prcp', 'slp','visib','max','min','sndp','wdsp','gust','dewp','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_mxpsd = df_mxpsd.loc[df_mxpsd["year"] != 2012]
df_mxpsd = df_mxpsd.loc[df_mxpsd["year"] < 2020]
cols = df_mxpsd['NUM_COLLISIONS']
df_mxpsd = df_mxpsd.drop(columns=['NUM_COLLISIONS'])
df_mxpsd.insert(loc=5, column='NUM_COLLISIONS', value=cols)
print(df_mxpsd[:6])
df_mxpsd.describe()

    day  year  mo  da  mxpsd  NUM_COLLISIONS
49    4  2016   1  28    8.9             681
51    5  2014   1  17    8.9             589
54    1  2016   1  25    8.9             658
55    5  2016   1  29    9.9             645
58    5  2017   1  20    9.9             605
59    7  2013   1  13    9.9             373


,day,year,mo,da,mxpsd,NUM_COLLISIONS
count,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000
mean,3.999608,2016.001567,6.520564,15.737172,17.240110,599.033686
std,2.001469,2.000587,3.449204,8.797367,5.858333,100.284761
min,1.000000,2013.000000,1.000000,1.000000,5.100000,188.000000
25%,2.000000,2014.000000,4.000000,8.000000,13.000000,531.000000
50%,4.000000,2016.000000,7.000000,16.000000,15.900000,602.000000
75%,6.000000,2018.000000,10.000000,23.000000,20.000000,665.000000
max,7.000000,2019.000000,12.000000,31.000000,49.000000,1161.000000


In [33]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_mxpsd.iloc[np.random.permutation(len(df_mxpsd))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]


# print out the first 6 rows of predictors.
print(predictors[:6])
print(shuffle[:6])

      day  year  mo  da  mxpsd
1024    1  2017   4  17   15.9
2919    7  2016  10  30   15.9
2501    1  2017   9  25    8.0
2581    1  2013   9  16   13.0
3503    3  2017  12  27   15.9
2795    2  2013  10  22    8.9
      day  year  mo  da  mxpsd  NUM_COLLISIONS
1024    1  2017   4  17   15.9             488
2919    7  2016  10  30   15.9             557
2501    1  2017   9  25    8.0             726
2581    1  2013   9  16   13.0             570
3503    3  2017  12  27   15.9             577
2795    2  2013  10  22    8.9             540


In [34]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

1024    488
2919    557
2501    726
2581    570
3503    577
2795    540
Name: NUM_COLLISIONS, dtype: int64


In [35]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
print(trainsize)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize
print(testsize)
# Define the number of input values (predictors)
nppredictors = 5
# Define the number of output values (targets)
noutputs = 1

2042
511


In [36]:
# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model_mxpsd', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_mxpsd', optimizer=tf.train.AdamOptimizer(learning_rate=0.00001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))
print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)
preds = estimator.predict(x=predictors[trainsize:].values)
#print(preds)
predslistscale = preds['scores']*SCALE_COLLISIONS

pred = format(str(predslistscale))
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff06316c990>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_mxpsd', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Create CheckpointSaverHook.


starting to train


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/linear_regression_trained_model_mxpsd/model.ckpt.
INFO:tensorflow:loss = 0.27086252, step = 1
INFO:tensorflow:global_step/sec: 1239.45
INFO:tensorflow:loss = 0.007692973, step = 101 (0.086 sec)
INFO:tensorflow:global_step/sec: 1386.65
INFO:tensorflow:loss = 0.0068669785, step = 201 (0.068 sec)
INFO:tensorflow:global_step/sec: 1531.29
INFO:tensorflow:loss = 0.0071460498, step = 301 (0.067 sec)
INFO:tensorflow:global_step/sec: 1500.44
INFO:tensorflow:loss = 0.0064879535, step = 401 (0.065 sec)
INFO:tensorflow:global_step/sec: 1469.24
INFO:tensorflow:loss = 0.007320543, step = 501 (0.068 sec)
INFO:tensorflow:global_step/sec: 1361.6
INFO:tensorflow:loss = 0.008667931, step = 601 (0.076 sec)
INFO:tensorflow:global_step/sec: 1337.09
INFO:tensorflow:loss = 0.006943797, step = 701 (0.073 sec)
INFO:tensorflow:global_step/sec: 

LinearRegression has RMSE of 90.92124313307619
Just using average = 598.3687561214496 has RMSE of 95.58877737762168


In [37]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_mxpsd', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff063076a10>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_mxpsd', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_mx

511
511
[0.55664074 0.515768   0.5580088  0.46999112 0.49476013 0.5406565
 0.506726   0.5268847  0.5087789  0.4979605  0.5219636  0.4720591
 0.55834895 0.5423378  0.52125597 0.5383248  0.5123896  0.5168648
 0.5671846  0.5617724  0.53588825 0.5535562  0.53042525 0.5536596
 0.4828721  0.49335313 0.5178631  0.49463364 0.47945145 0.48343104
 0.48866022 0.5546254  0.5399067  0.5019694  0.48999566 0.539752
 0.5393192  0.4653684  0.5555799  0.4643746  0.47413298 0.4862343
 0.5030523  0.5148423  0.5228981  0.5655355  0.5220009  0.5613953
 0.52137446 0.5192911  0.5320688  0.5040822  0.565997   0.47489133
 0.46501958 0.5415519  0.57073015 0.5554183  0.4726756  0.5236754
 0.51009387 0.5524381  0.519096   0.55840987 0.5715418  0.48918727
 0.5508579  0.5149744  0.47929227 0.4947867  0.5101786  0.50774735
 0.52697563 0.5017576  0.55243886 0.5287167  0.49873173 0.5275012
 0.4936037  0.4891511  0.5496686  0.51515156 0.5254449  0.4812897
 0.5456325  0.53978205 0.53203857 0.5566334  0.5521704  0.5171166

##Others


#Deep Learning Neural Network (DNN)

####Precipition (prcp)

In [38]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/prcp_clean_dnn.csv', index_col=0, )
print(df[:6])

   year  da  NUM_COLLISIONS  temp  dewp     slp  visib  wdsp  mxpsd   gust  \
1  2020  24             524  37.3  33.7  1028.5    6.5   3.3    8.0  999.9   
2  2021  12             278  37.0  29.1  1019.0   10.0   6.5   12.0  999.9   
3  2021  22             254  36.5  28.4  1003.1   10.0   7.8   12.0   20.0   
4  2021  27             262  34.6  33.8  1012.8    8.0   7.8   12.0  999.9   
5  2021  26             263  31.9  23.4  1016.9    9.0   7.4   12.0  999.9   
6  2022  24             237  34.5  23.8  1010.6    9.7   7.4   12.0  999.9   

   ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1  ...    0    0    0    0    0    0    0    1    0    0  
2  ...    0    0    0    0    1    0    0    0    0    0  
3  ...    0    0    0    0    0    0    0    1    0    0  
4  ...    0    0    0    0    0    0    0    0    1    0  
5  ...    0    0    0    0    1    0    0    0    0    0  
6  ...    0    0    0    0    0    0    1    0    0    0  

[6 rows x 38 columns]


In [39]:
df_prcp_dnn = df.drop(columns=['temp', 'dewp', 'slp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud'])
df_prcp_dnn = df_prcp_dnn.loc[df_prcp_dnn["year"] != 2012]
df_prcp_dnn = df_prcp_dnn.loc[df_prcp_dnn["year"] < 2020]
cols = df_prcp_dnn['NUM_COLLISIONS']
df_prcp_dnn = df_prcp_dnn.drop(columns=['NUM_COLLISIONS'])
df_prcp_dnn.insert(loc=25, column='NUM_COLLISIONS', value=cols)
print(df_prcp_dnn[:6])
df_prcp_dnn.describe()

    year  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail  Apr  Aug  Dec  \
49  2016  28  0.09    0             0                 0     0    0    0    0   
51  2014  17  0.00    1             0                 0     0    0    0    0   
54  2016  25  0.02    0             0                 0     0    0    0    0   
55  2016  29  0.00    0             0                 0     0    0    0    0   
58  2017  20  0.00    0             0                 0     0    0    0    0   
59  2013  13  0.01    1             0                 0     0    0    0    0   

    ...  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
49  ...    0    0    0    0    0    0    0    0    1             681  
51  ...    0    0    0    0    0    0    1    0    0             589  
54  ...    0    0    0    0    0    1    0    0    0             658  
55  ...    0    0    0    0    0    0    1    0    0             645  
58  ...    0    0    0    0    0    0    1    0    0             605  
59  ...    0 

,year,da,prcp,fog,rain_drizzle,snow_ice_pellets,hail,Apr,Aug,Dec,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,...,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000
mean,2015.989366,15.745569,0.122588,0.253249,0.375345,0.085467,0.000394,0.082316,0.083497,0.085467,...,0.082710,0.085467,0.079953,0.142970,0.143364,0.143757,0.142182,0.142576,0.142182,599.135093
std,1.996126,8.803199,0.329143,0.434958,0.484307,0.279630,0.019846,0.274899,0.276687,0.279630,...,0.275497,0.279630,0.271273,0.350111,0.350512,0.350913,0.349305,0.349709,0.349305,100.299164
min,2013.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2014.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000
50%,2016.000000,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000
75%,2018.000000,23.000000,0.060000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,665.000000
max,2019.000000,31.000000,3.760000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [40]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_prcp_dnn.iloc[np.random.permutation(len(df_prcp_dnn))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

      year  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail  Apr  Aug  \
2213  2016  30  0.00    0             0                 0     0    0    1   
275   2017  11  0.33    0             1                 0     0    0    0   
650   2014   1  0.00    0             0                 0     0    0    0   
3525  2017   7  0.02    0             0                 0     0    0    0   
642   2018  23  0.07    0             0                 0     0    0    0   
1057  2014   9  0.45    0             0                 0     0    1    0   

      Dec  ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
2213    0  ...    0    0    0    0    1    0    0    0    0    0  
275     0  ...    0    0    0    0    0    0    0    0    1    0  
650     0  ...    0    0    0    0    0    0    0    0    0    0  
3525    1  ...    0    0    0    0    0    0    0    0    0    1  
642     0  ...    0    0    0    0    0    0    0    1    0    0  
1057    0  ...    0    0    0    0    0    0    0    0    

In [41]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

2213    622
275     629
650     497
3525    693
642     732
1057    595
Name: NUM_COLLISIONS, dtype: int64


In [42]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

# Define the number of output values (targets)
noutputs = 1
# Define the number of input values (predictors)
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)

25


In [43]:

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model_prcp', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_prcp', hidden_units=[20,18,13], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_COLLISIONS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff06326e250>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_prcp', '_session_creation_timeout_secs': 7200}
Instructions for updating:
Please use `layer.__call__` method instead.


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model_prcp/model.ckpt.
INFO:tensorflow:loss = 125586.39, step = 1
INFO:tensorflow:global_step/sec: 732.861
INFO:tensorflow:loss = 0.21339825, step = 101 (0.141 sec)
INFO:tensorflow:global_step/sec: 965.989
INFO:tensorflow:loss = 0.1458404, step = 201 (0.104 sec)
INFO:tensorflow:global_step/sec: 921.26
INFO:tensorflow:loss = 0.1365065, step = 301 (0.109 sec)
INFO:tensorflow:global_step/sec: 930.901
INFO:tensorflow:loss = 0.14307836, step = 401 (0.105 sec)
INFO:tensorflow:global_step/sec: 987.158
INFO:tensorflow:loss = 0.11546454, step = 501 (0.103 sec)
INFO:tensorflow:global_step/sec: 1037.87
INFO:tensorflow:loss = 0.08977402, step = 601 (0.097 sec)
INFO:tensorflow:global_step/sec: 1043.4
INFO:tensorflow:loss = 0.09675571, step = 701 (0.096 sec)
INFO:te

DNNRegression has RMSE of 84.84903754233527
Just using average = 598.8050221565732 has RMSE of 97.02374587140838


In [44]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_prcp', hidden_units=[20,18,13], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff062e31b50>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_prcp', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_prcp/mode

508
508


INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.53398955 0.54161865 0.5170258  0.41488212 0.51392347 0.52405334
 0.47674736 0.50921977 0.48811716 0.5183796  0.5045387  0.5307376
 0.5018388  0.47520146 0.5100182  0.54444134 0.50718445 0.4675887
 0.563306   0.47968167 0.50087845 0.409858   0.55063665 0.5247508
 0.48154104 0.5181476  0.55130076 0.4897993  0.5553692  0.47166947
 0.52934533 0.53669703 0.5125815  0.4756523  0.53374577 0.54030013
 0.5424858  0.5092124  0.51411724 0.508694   0.49558157 0.5191568
 0.54637694 0.49556643 0.52535594 0.5701045  0.5116277  0.45969743
 0.511182   0.50846064 0.56613135 0.5011097  0.43986717 0.4627719
 0.5160634  0.5286639  0.52235925 0.52916133 0.40699974 0.47095713
 0.4797127  0.5099393  0.555771   0.53634465 0.5245134  0.4553373
 0.53089696 0.48551467 0.4686514  0.55081064 0.5404421  0.48438203
 0.48307312 0.5444004  0.5264542  0.51868147 0.5063669  0.48636115
 0.54961365 0.47714847 0.5047677  0.5082794  0.5321127  0.49253583
 0.46418363 0.48944935 0.5011878  0.46203455 0.4991766  0.43332943
 

##Dew Point (dewp)


In [45]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/dewp_clean_dnn.csv', index_col=0, )
print(df[:6])

   year  da  NUM_COLLISIONS  temp  dewp     slp  visib  wdsp  mxpsd   gust  \
1  2020  24             524  37.3  33.7  1028.5    6.5   3.3    8.0  999.9   
2  2021  12             278  37.0  29.1  1019.0   10.0   6.5   12.0  999.9   
3  2021  22             254  36.5  28.4  1003.1   10.0   7.8   12.0   20.0   
4  2021  27             262  34.6  33.8  1012.8    8.0   7.8   12.0  999.9   
5  2021  26             263  31.9  23.4  1016.9    9.0   7.4   12.0  999.9   
6  2022  24             237  34.5  23.8  1010.6    9.7   7.4   12.0  999.9   

   ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1  ...    0    0    0    0    0    0    0    1    0    0  
2  ...    0    0    0    0    1    0    0    0    0    0  
3  ...    0    0    0    0    0    0    0    1    0    0  
4  ...    0    0    0    0    0    0    0    0    1    0  
5  ...    0    0    0    0    1    0    0    0    0    0  
6  ...    0    0    0    0    0    0    1    0    0    0  

[6 rows x 38 columns]


In [46]:
df_dewp_dnn = df.drop(columns=['temp', 'prcp', 'slp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_dewp_dnn = df_dewp_dnn.loc[df_dewp_dnn["year"] != 2012]
df_dewp_dnn = df_dewp_dnn.loc[df_dewp_dnn["year"] < 2020]
cols = df_dewp_dnn['NUM_COLLISIONS']
df_dewp_dnn = df_dewp_dnn.drop(columns=['NUM_COLLISIONS'])
df_dewp_dnn.insert(loc=21, column='NUM_COLLISIONS', value=cols)
print(df_dewp_dnn[:6])
df_dewp_dnn.describe()

    year  da  dewp  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  Nov  Oct  Sep  \
49  2016  28  24.4    0    0    0    0    1    0    0  ...    0    0    0   
51  2014  17  35.8    0    0    0    0    1    0    0  ...    0    0    0   
54  2016  25  21.2    0    0    0    0    1    0    0  ...    0    0    0   
55  2016  29  36.8    0    0    0    0    1    0    0  ...    0    0    0   
58  2017  20  32.5    0    0    0    0    1    0    0  ...    0    0    0   
59  2013  13  44.9    0    0    0    0    1    0    0  ...    0    0    0   

    Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
49    0    0    0    0    0    1             681  
51    0    0    0    1    0    0             589  
54    0    0    1    0    0    0             658  
55    0    0    0    1    0    0             645  
58    0    0    0    1    0    0             605  
59    0    1    0    0    0    0             373  

[6 rows x 22 columns]


,year,da,dewp,Apr,Aug,Dec,Feb,Jan,Jul,Jun,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,...,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000
mean,2015.999217,15.723679,44.163170,0.082192,0.084932,0.084932,0.077104,0.084932,0.084540,0.082192,...,0.082192,0.084932,0.082192,0.143249,0.142857,0.142857,0.142857,0.142857,0.142857,599.109980
std,2.000000,8.801271,16.995303,0.274710,0.278834,0.278834,0.266808,0.278834,0.278251,0.274710,...,0.274710,0.278834,0.274710,0.350395,0.349996,0.349996,0.349996,0.349996,0.349996,100.277185
min,2013.000000,1.000000,-6.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2014.000000,8.000000,32.150000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000
50%,2016.000000,16.000000,45.300000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000
75%,2018.000000,23.000000,58.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,665.000000
max,2019.000000,31.000000,74.100000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [47]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_dewp_dnn.iloc[np.random.permutation(len(df_dewp_dnn))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

      year  da  dewp  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  May  Nov  Oct  \
2445  2019  21  68.5    0    1    0    0    0    0    0  ...    0    0    0   
1121  2013  23  40.0    1    0    0    0    0    0    0  ...    0    0    0   
505   2016   4  49.4    0    0    0    1    0    0    0  ...    0    0    0   
801   2014  22  27.9    0    0    0    0    0    0    0  ...    0    0    0   
2647  2014   8  55.3    0    0    0    0    0    0    0  ...    0    0    0   
843   2018  27  28.7    0    0    0    0    0    0    0  ...    0    0    0   

      Sep  Mon  Sat  Sun  Thu  Tue  Wed  
2445    0    0    0    0    0    1    0  
1121    0    1    0    0    0    0    0  
505     0    0    0    0    0    0    1  
801     0    0    0    0    0    0    0  
2647    1    0    0    1    0    0    0  
843     0    1    0    0    0    0    0  

[6 rows x 21 columns]


In [48]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

2445    522
1121    513
505     609
801     493
2647    598
843     627
Name: NUM_COLLISIONS, dtype: int64


In [49]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

# Define the number of output values (targets)
noutputs = 1
# Define the number of input values (predictors)
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)

21


In [50]:

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model_dewp', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_dewp', hidden_units=[20,18,13], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_COLLISIONS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff062d8f050>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_dewp', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model_dewp/model.ckpt.
INFO:tensorflow:loss = 41988.633, step = 1
INFO:tensorflow:global_step/sec: 794.948
INFO:tensorflow:loss = 0.896653, step = 101 (0.130 sec)
INFO:tensorflow:global_step/sec: 1019.97
INFO:tensorflow:loss = 0.77257544, step = 201 (0.099 sec)
INFO:tensorflow:global_step/sec: 1045.91
INFO:tensorflow:loss = 0.45631158, step = 301 (0.093 sec)
INFO:tensorflow:global_step/sec: 991.182
INFO:tensorflow:loss = 0.41470122, step = 401 (0.104 sec)
INFO:tensorflow:global_step/sec: 945.174
INFO:tensorflow:loss = 0.2373845, step = 501 (0.104 sec)
INFO:tensorflow:global_step/sec: 1003.41
INFO:tensorflow:loss = 0.16055687, step = 601 (0.100 sec)
INFO:tensorflow:global_step/sec: 943.35
INFO:tensorflow:loss = 0.1007534, step = 701 (0.104 sec)
INFO:ten

DNNRegression has RMSE of 111.0434696557508
Just using average = 598.8840508806262 has RMSE of 98.9618419937131


In [51]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_dewp', hidden_units=[20,18,13], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff05f648d90>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_dewp', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_dewp/mode

511
511


INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.47293308 0.4661208  0.50386965 0.47377914 0.5409767  0.4247111
 0.35220084 0.49011683 0.48777962 0.5428697  0.42196664 0.31290406
 0.43790182 0.44153386 0.48399454 0.43879056 0.44979125 0.41787437
 0.42273122 0.37880167 0.45410633 0.4766802  0.3464664  0.44413587
 0.3446467  0.44061399 0.42643547 0.4920067  0.4744917  0.43529135
 0.47771668 0.4854771  0.5136565  0.41682816 0.40898955 0.3742489
 0.4680891  0.3766319  0.43941024 0.4730195  0.33534604 0.43892264
 0.46058774 0.46027243 0.42426816 0.47786957 0.4327618  0.3837127
 0.47326058 0.48072028 0.43163693 0.42435777 0.34137136 0.50533414
 0.4866422  0.4863485  0.5170871  0.4987314  0.44671232 0.3687474
 0.43214214 0.35221758 0.46011615 0.46182477 0.4895174  0.41493767
 0.41424352 0.38315883 0.48386434 0.4490556  0.5347531  0.4336978
 0.47816873 0.46794328 0.41635808 0.45023856 0.4972243  0.4410442
 0.46587542 0.39563453 0.4753736  0.34190145 0.39143208 0.3983619
 0.43861973 0.45278952 0.50985265 0.42010576 0.44087103 0.45072493
 0

##Sea Level Pressure(slp)


In [52]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/slp_clean_dnn.csv', index_col=0, )
print(df[:6])

   year  da  NUM_COLLISIONS  temp  dewp     slp  visib  wdsp  mxpsd   gust  \
1  2020  24             524  37.3  33.7  1028.5    6.5   3.3    8.0  999.9   
2  2021  12             278  37.0  29.1  1019.0   10.0   6.5   12.0  999.9   
3  2021  22             254  36.5  28.4  1003.1   10.0   7.8   12.0   20.0   
4  2021  27             262  34.6  33.8  1012.8    8.0   7.8   12.0  999.9   
5  2021  26             263  31.9  23.4  1016.9    9.0   7.4   12.0  999.9   
6  2022  24             237  34.5  23.8  1010.6    9.7   7.4   12.0  999.9   

   ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1  ...    0    0    0    0    0    0    0    1    0    0  
2  ...    0    0    0    0    1    0    0    0    0    0  
3  ...    0    0    0    0    0    0    0    1    0    0  
4  ...    0    0    0    0    0    0    0    0    1    0  
5  ...    0    0    0    0    1    0    0    0    0    0  
6  ...    0    0    0    0    0    0    1    0    0    0  

[6 rows x 38 columns]


In [53]:
df_slp_dnn = df.drop(columns=['temp', 'prcp', 'dewp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_slp_dnn = df_slp_dnn.loc[df_slp_dnn["year"] != 2012]
df_slp_dnn = df_slp_dnn.loc[df_slp_dnn["year"] < 2020]
cols = df_slp_dnn['NUM_COLLISIONS']
df_slp_dnn = df_slp_dnn.drop(columns=['NUM_COLLISIONS'])
df_slp_dnn.insert(loc=21, column='NUM_COLLISIONS', value=cols)
print(df_slp_dnn[:6])
df_slp_dnn.describe()

    year  da     slp  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  Nov  Oct  Sep  \
49  2016  28  1016.1    0    0    0    0    1    0    0  ...    0    0    0   
51  2014  17  1014.8    0    0    0    0    1    0    0  ...    0    0    0   
54  2016  25  1021.4    0    0    0    0    1    0    0  ...    0    0    0   
55  2016  29   999.4    0    0    0    0    1    0    0  ...    0    0    0   
58  2017  20  1015.5    0    0    0    0    1    0    0  ...    0    0    0   
59  2013  13  1020.7    0    0    0    0    1    0    0  ...    0    0    0   

    Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
49    0    0    0    0    0    1             681  
51    0    0    0    1    0    0             589  
54    0    0    1    0    0    0             658  
55    0    0    0    1    0    0             645  
58    0    0    0    1    0    0             605  
59    0    1    0    0    0    0             373  

[6 rows x 22 columns]


,year,da,slp,Apr,Aug,Dec,Feb,Jan,Jul,Jun,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,...,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000
mean,2016.000391,15.719765,1016.777221,0.082192,0.084932,0.084540,0.077104,0.084932,0.084932,0.082192,...,0.082192,0.084932,0.082192,0.143249,0.142857,0.142857,0.142857,0.142857,0.142466,599.147162
std,2.000294,8.796698,7.628429,0.274710,0.278834,0.278251,0.266808,0.278834,0.278834,0.274710,...,0.274710,0.278834,0.274710,0.350395,0.349996,0.349996,0.349996,0.349996,0.349596,100.268048
min,2013.000000,1.000000,989.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2014.000000,8.000000,1012.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000
50%,2016.000000,16.000000,1016.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000
75%,2018.000000,23.000000,1021.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,665.000000
max,2019.000000,31.000000,1044.200000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [54]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_slp_dnn.iloc[np.random.permutation(len(df_slp_dnn))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

      year  da     slp  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  May  Nov  Oct  \
635   2019   1  1026.9    0    0    0    0    0    0    0  ...    0    0    0   
383   2017  28  1029.7    0    0    0    1    0    0    0  ...    0    0    0   
3662  2019   2   996.1    0    0    1    0    0    0    0  ...    0    0    0   
2130  2017  13  1015.5    0    0    0    0    0    1    0  ...    0    0    0   
1681  2017  22  1014.3    0    0    0    0    0    0    1  ...    0    0    0   
2579  2017  13  1011.8    0    0    0    0    0    0    0  ...    0    0    0   

      Sep  Mon  Sat  Sun  Thu  Tue  Wed  
635     0    0    0    0    1    0    0  
383     0    1    0    0    0    0    0  
3662    0    0    0    1    0    0    0  
2130    0    0    0    0    0    0    1  
1681    0    0    0    0    0    0    1  
2579    1    0    0    0    0    1    0  

[6 rows x 21 columns]


In [55]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

635     711
383     591
3662    580
2130    759
1681    845
2579    738
Name: NUM_COLLISIONS, dtype: int64


In [56]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

# Define the number of output values (targets)
noutputs = 1
# Define the number of input values (predictors)
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)

21


In [57]:

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model_slp', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_slp', hidden_units=[19,15,11], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_COLLISIONS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff05f55ffd0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_slp', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model_slp/model.ckpt.
INFO:tensorflow:loss = 509618.3, step = 1
INFO:tensorflow:global_step/sec: 517.174
INFO:tensorflow:loss = 0.57567465, step = 101 (0.199 sec)
INFO:tensorflow:global_step/sec: 699.395
INFO:tensorflow:loss = 0.106411755, step = 201 (0.141 sec)
INFO:tensorflow:global_step/sec: 664.54
INFO:tensorflow:loss = 0.11206909, step = 301 (0.151 sec)
INFO:tensorflow:global_step/sec: 670.819
INFO:tensorflow:loss = 0.10552477, step = 401 (0.151 sec)
INFO:tensorflow:global_step/sec: 668.368
INFO:tensorflow:loss = 0.08431721, step = 501 (0.146 sec)
INFO:tensorflow:global_step/sec: 669.738
INFO:tensorflow:loss = 0.10270215, step = 601 (0.158 sec)
INFO:tensorflow:global_step/sec: 645.414
INFO:tensorflow:loss = 0.09978469, step = 701 (0.149 sec)
INFO:

DNNRegression has RMSE of 325.9053488231647
Just using average = 599.6526418786693 has RMSE of 97.54128152017972


In [58]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_slp', hidden_units=[19,15,11], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff0633adc90>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_slp', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_slp/model.

511
511


INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.7650063  0.85170496 0.71542037 0.8176734  0.8068422  0.8713988
 0.8307384  0.81752706 0.8352272  0.85883594 0.7978668  0.77376044
 0.6616859  0.80041957 0.8630664  0.83913916 0.8236946  0.70176065
 0.74473643 0.77849674 0.7872642  0.80802774 0.82792807 0.7610702
 0.8308375  0.74603474 0.8560032  0.6451653  0.7078787  0.7178435
 0.81761664 0.7678099  0.8876463  0.68051    0.7846068  0.82772434
 0.8339045  0.7605138  0.8323385  0.81982964 0.8397672  0.7170262
 0.81011254 0.81248116 0.75252414 0.767634   0.70598274 0.86346376
 0.84157044 0.7962547  0.8030907  0.8396168  0.84075147 0.73308915
 0.79712987 0.8009829  0.76609534 0.845349   0.7811843  0.90376514
 0.8302456  0.7032007  0.7854685  0.78940177 0.7246854  0.8118154
 0.8267008  0.87193406 0.7947912  0.7421315  0.78726    0.80716985
 0.79577297 0.7920041  0.7779674  0.75607795 0.7794394  0.8437662
 0.77941066 0.81955916 0.8080253  0.86207616 0.7811331  0.77929676
 0.82407117 0.6843045  0.6945485  0.7932344  0.84839094 0.75510675
 

##Gust

In [59]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/gust_clean_dnn.csv', index_col=0, )
print(df[:6])

    year  da  NUM_COLLISIONS  temp  dewp     slp  visib  wdsp  mxpsd  gust  \
3   2021  22             254  36.5  28.4  1003.1   10.0   7.8   12.0  20.0   
11  2020  15             508  43.9  38.3  1019.4    8.2   5.4   14.0  15.0   
12  2021   1             257  39.6  29.3  1029.3   10.0   7.6   14.0  20.0   
14  2022  25             235  41.6  31.8  1013.2   10.0   9.6   15.0  19.0   
18  2021   3             186  41.1  32.3  1018.0   10.0  10.3   19.0  27.0   
19  2020   2             413  39.6  28.9  1011.8   10.0  13.0   19.0  26.0   

    ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
3   ...    0    0    0    0    0    0    0    1    0    0  
11  ...    0    0    0    0    0    0    0    0    1    0  
12  ...    0    0    0    0    0    0    0    1    0    0  
14  ...    0    0    0    0    1    0    0    0    0    0  
18  ...    0    0    0    0    0    1    0    0    0    0  
19  ...    0    0    0    0    0    0    0    0    0    1  

[6 rows x 38 columns]


In [60]:
df_gust_dnn = df.drop(columns=['temp', 'prcp', 'dewp','visib','max','min','sndp','wdsp','mxpsd','slp','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_gust_dnn = df_gust_dnn.loc[df_gust_dnn["year"] != 2012]
df_gust_dnn = df_gust_dnn.loc[df_gust_dnn["year"] < 2020]
cols = df_gust_dnn['NUM_COLLISIONS']
df_gust_dnn = df_gust_dnn.drop(columns=['NUM_COLLISIONS'])
df_gust_dnn.insert(loc=21, column='NUM_COLLISIONS', value=cols)
print(df_gust_dnn[:6])
df_gust_dnn.describe()

    year  da  gust  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  Nov  Oct  Sep  \
74  2016  17  18.1    0    0    0    0    1    0    0  ...    0    0    0   
76  2014   9  20.0    0    0    0    0    1    0    0  ...    0    0    0   
79  2019  19  21.0    0    0    0    0    1    0    0  ...    0    0    0   
80  2015  11  17.1    0    0    0    0    1    0    0  ...    0    0    0   
83  2015  29  20.0    0    0    0    0    1    0    0  ...    0    0    0   
85  2019  13  15.9    0    0    0    0    1    0    0  ...    0    0    0   

    Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
74    0    1    0    0    0    0             451  
76    0    0    0    0    0    1             561  
79    0    0    0    0    0    0             479  
80    0    1    0    0    0    0             341  
83    0    0    0    0    0    1             519  
85    0    1    0    0    0    0             374  

[6 rows x 22 columns]


,year,da,gust,Apr,Aug,Dec,Feb,Jan,Jul,Jun,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,1629.00000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,...,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000
mean,2015.91283,15.702885,27.511602,0.095764,0.042357,0.104359,0.095150,0.108656,0.046041,0.061387,...,0.096378,0.087784,0.071209,0.139963,0.141191,0.139963,0.151627,0.138122,0.145488,596.513198
std,2.01341,8.667634,7.366770,0.294358,0.201465,0.305819,0.293513,0.311302,0.209637,0.240113,...,0.295200,0.283067,0.257253,0.347055,0.348325,0.347055,0.358769,0.345133,0.352700,104.479660
min,2013.00000,1.000000,14.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2014.00000,8.000000,22.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,526.000000
50%,2016.00000,16.000000,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,597.000000
75%,2018.00000,23.000000,31.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,663.000000
max,2019.00000,31.000000,71.100000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [61]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_gust_dnn.iloc[np.random.permutation(len(df_gust_dnn))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

      year  da  gust  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  May  Nov  Oct  \
1792  2013  19  28.0    0    0    0    0    0    0    1  ...    0    0    0   
302   2016  10  41.0    0    0    0    0    1    0    0  ...    0    0    0   
1132  2016  10  28.9    1    0    0    0    0    0    0  ...    0    0    0   
295   2019  24  42.0    0    0    0    0    1    0    0  ...    0    0    0   
3029  2017  31  29.9    0    0    0    0    0    0    0  ...    0    0    1   
3459  2017  31  19.0    0    0    1    0    0    0    0  ...    0    0    0   

      Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1792    0    0    0    0    0    1    0  
302     0    0    1    0    0    0    0  
1132    0    0    1    0    0    0    0  
295     0    0    0    0    0    0    1  
3029    0    1    0    0    0    0    0  
3459    0    0    1    0    0    0    0  

[6 rows x 21 columns]


In [62]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

1792    622
302     462
1132    499
295     592
3029    696
3459    462
Name: NUM_COLLISIONS, dtype: int64


In [63]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

# Define the number of output values (targets)
noutputs = 1
# Define the number of input values (predictors)
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)

21


In [64]:

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model_gust', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_gust', hidden_units=[23,19,11], optimizer=tf.train.AdamOptimizer(learning_rate=0.001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_COLLISIONS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff0633ad250>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_gust', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model_gust/model.ckpt.
INFO:tensorflow:loss = 64.971466, step = 1
INFO:tensorflow:global_step/sec: 719.654
INFO:tensorflow:loss = 0.053713143, step = 101 (0.143 sec)
INFO:tensorflow:global_step/sec: 808.333
INFO:tensorflow:loss = 0.07465811, step = 201 (0.124 sec)
INFO:tensorflow:global_step/sec: 926.883
INFO:tensorflow:loss = 0.04166411, step = 301 (0.108 sec)
INFO:tensorflow:global_step/sec: 1032.47
INFO:tensorflow:loss = 0.03414039, step = 401 (0.097 sec)
INFO:tensorflow:global_step/sec: 997.067
INFO:tensorflow:loss = 0.0359189, step = 501 (0.100 sec)
INFO:tensorflow:global_step/sec: 969.441
INFO:tensorflow:loss = 0.023819624, step = 601 (0.102 sec)
INFO:tensorflow:global_step/sec: 940.353
INFO:tensorflow:loss = 0.01677313, step = 701 (0.105 sec)
IN

DNNRegression has RMSE of 109.47309609016882
Just using average = 594.3284727551803 has RMSE of 112.76313210041813


In [65]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_gust', hidden_units=[23,19,11], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff06551a890>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_gust', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_gust/mode

326
326


INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.5453121  0.60026455 0.6028755  0.4785436  0.5091958  0.56914186
 0.45735314 0.62659085 0.61591715 0.61809313 0.6044074  0.6023531
 0.5059775  0.5864702  0.54461575 0.59787965 0.46467242 0.5054646
 0.5726779  0.5931157  0.54931164 0.6487645  0.62044567 0.48291382
 0.57112676 0.49956048 0.59300834 0.6160402  0.60995895 0.5823898
 0.5824373  0.6284571  0.60100967 0.5356183  0.687197   0.42860606
 0.61914915 0.5985282  0.66340464 0.6110858  0.6481132  0.5683931
 0.6199402  0.500204   0.49694598 0.5678619  0.59557885 0.48710504
 0.5841349  0.5850111  0.5249624  0.60065144 0.53344035 0.52301383
 0.5369082  0.5610239  0.55145556 0.66302645 0.6240687  0.6524092
 0.60970265 0.5323507  0.5642173  0.56589836 0.5918983  0.57658297
 0.58732563 0.6114651  0.6243283  0.5540429  0.6111728  0.62210685
 0.5409451  0.5999031  0.5043643  0.60960186 0.5706019  0.4779831
 0.54000986 0.6122121  0.5391625  0.55473936 0.6238618  0.60633
 0.62376064 0.57474834 0.56689227 0.5489578  0.5791922  0.5475176
 0.64

##Maximum Sustained Wind Speed (mxpsd)

In [66]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/mxpsd_clean_dnn.csv', index_col=0, )
print(df[:6])

   year  da  NUM_COLLISIONS  temp  dewp     slp  visib  wdsp  mxpsd   gust  \
1  2020  24             524  37.3  33.7  1028.5    6.5   3.3    8.0  999.9   
2  2021  12             278  37.0  29.1  1019.0   10.0   6.5   12.0  999.9   
3  2021  22             254  36.5  28.4  1003.1   10.0   7.8   12.0   20.0   
4  2021  27             262  34.6  33.8  1012.8    8.0   7.8   12.0  999.9   
5  2021  26             263  31.9  23.4  1016.9    9.0   7.4   12.0  999.9   
6  2022  24             237  34.5  23.8  1010.6    9.7   7.4   12.0  999.9   

   ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1  ...    0    0    0    0    0    0    0    1    0    0  
2  ...    0    0    0    0    1    0    0    0    0    0  
3  ...    0    0    0    0    0    0    0    1    0    0  
4  ...    0    0    0    0    0    0    0    0    1    0  
5  ...    0    0    0    0    1    0    0    0    0    0  
6  ...    0    0    0    0    0    0    1    0    0    0  

[6 rows x 38 columns]


In [67]:
df_mxpsd_dnn = df.drop(columns=['temp', 'prcp', 'dewp','visib','max','min','sndp','wdsp','gust','slp','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_mxpsd_dnn = df_mxpsd_dnn.loc[df_mxpsd_dnn["year"] != 2012]
df_mxpsd_dnn = df_mxpsd_dnn.loc[df_mxpsd_dnn["year"] < 2020]
cols = df_mxpsd_dnn['NUM_COLLISIONS']
df_mxpsd_dnn = df_mxpsd_dnn.drop(columns=['NUM_COLLISIONS'])
df_mxpsd_dnn.insert(loc=21, column='NUM_COLLISIONS', value=cols)
print(df_mxpsd_dnn[:6])
df_mxpsd_dnn.describe()

    year  da  mxpsd  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  Nov  Oct  Sep  \
49  2016  28    8.9    0    0    0    0    1    0    0  ...    0    0    0   
51  2014  17    8.9    0    0    0    0    1    0    0  ...    0    0    0   
54  2016  25    8.9    0    0    0    0    1    0    0  ...    0    0    0   
55  2016  29    9.9    0    0    0    0    1    0    0  ...    0    0    0   
58  2017  20    9.9    0    0    0    0    1    0    0  ...    0    0    0   
59  2013  13    9.9    0    0    0    0    1    0    0  ...    0    0    0   

    Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
49    0    0    0    0    0    1             681  
51    0    0    0    1    0    0             589  
54    0    0    1    0    0    0             658  
55    0    0    0    1    0    0             645  
58    0    0    0    1    0    0             605  
59    0    1    0    0    0    0             373  

[6 rows x 22 columns]


,year,da,mxpsd,Apr,Aug,Dec,Feb,Jan,Jul,Jun,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,...,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000
mean,2016.001567,15.737172,17.240110,0.082256,0.084998,0.084998,0.077164,0.084998,0.084998,0.082256,...,0.081864,0.084998,0.081473,0.143361,0.142969,0.142969,0.142969,0.142577,0.142186,599.033686
std,2.000587,8.797367,5.858333,0.274808,0.278933,0.278933,0.266904,0.278933,0.278933,0.274808,...,0.274212,0.278933,0.273613,0.350509,0.350110,0.350110,0.350110,0.349710,0.349309,100.284761
min,2013.000000,1.000000,5.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2014.000000,8.000000,13.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000
50%,2016.000000,16.000000,15.900000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000
75%,2018.000000,23.000000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,665.000000
max,2019.000000,31.000000,49.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [68]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_mxpsd_dnn.iloc[np.random.permutation(len(df_mxpsd_dnn))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

      year  da  mxpsd  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  May  Nov  Oct  \
1692  2013   4   15.0    0    0    0    0    0    0    1  ...    0    0    0   
3196  2017  14   17.1    0    0    0    0    0    0    0  ...    0    1    0   
493   2018   5   20.0    0    0    0    1    0    0    0  ...    0    0    0   
1373  2017  18   15.9    0    0    0    0    0    0    0  ...    1    0    0   
237   2014  11   22.9    0    0    0    0    1    0    0  ...    0    0    0   
3113  2017  24    9.9    0    0    0    0    0    0    0  ...    0    1    0   

      Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1692    0    1    0    0    0    0    0  
3196    0    1    0    0    0    0    0  
493     0    0    0    1    0    0    0  
1373    0    0    0    0    0    0    1  
237     0    0    0    0    0    0    0  
3113    0    0    0    0    1    0    0  

[6 rows x 21 columns]


In [69]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

1692    613
3196    647
493     706
1373    911
237     465
3113    502
Name: NUM_COLLISIONS, dtype: int64


In [70]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

# Define the number of output values (targets)
noutputs = 1
# Define the number of input values (predictors)
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)

21


In [71]:

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model_mxpsd', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_mxpsd', hidden_units=[19,15,11], optimizer=tf.train.AdamOptimizer(learning_rate=0.001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_COLLISIONS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff0631a0110>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_mxpsd', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model_mxpsd/model.ckpt.
INFO:tensorflow:loss = 85647.24, step = 1
INFO:tensorflow:global_step/sec: 534.607
INFO:tensorflow:loss = 1.0603168, step = 101 (0.190 sec)
INFO:tensorflow:global_step/sec: 690.448
INFO:tensorflow:loss = 0.5262594, step = 201 (0.147 sec)
INFO:tensorflow:global_step/sec: 683.89
INFO:tensorflow:loss = 0.5222115, step = 301 (0.147 sec)
INFO:tensorflow:global_step/sec: 668.151
INFO:tensorflow:loss = 0.58860123, step = 401 (0.148 sec)
INFO:tensorflow:global_step/sec: 673.646
INFO:tensorflow:loss = 0.52257407, step = 501 (0.150 sec)
INFO:tensorflow:global_step/sec: 655.22
INFO:tensorflow:loss = 0.428842, step = 601 (0.150 sec)
INFO:tensorflow:global_step/sec: 694.461
INFO:tensorflow:loss = 0.44657278, step = 701 (0.144 sec)
INFO:tenso

DNNRegression has RMSE of 81.16662150752263
Just using average = 600.0881488736533 has RMSE of 98.35068083241072


In [72]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_gust', hidden_units=[23,19,11], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff062d55d90>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_gust', '_session_creation_timeout_secs': 7200}


511
511


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_gust/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.5527696  0.52508855 0.6127819  0.581434   0.61828417 0.61175823
 0.63292027 0.59236956 0.5249682  0.5626297  0.5715749  0.60353154
 0.47396922 0.581      0.6268806  0.5770799  0.62899625 0.47361183
 0.6106329  0.56895494 0.5959188  0.6305055  0.62277585 0.5162532
 0.61939365 0.52758753 0.59395343 0.5659039  0.684877   0.681315
 0.5774379  0.6243287  0.57529247 0.5911565  0.47798577 0.4547477
 0.5292468  0.5924553  0.6320611  0.5995403  0.4729641  0.55078274
 0.5829641  0.49724042 0.60598487 0.5737326  0.62647694 0.61818373
 0.6239849  0.55638236 0.5590047  0.4733155  0.5410132  0.5791286
 0.6186458  0.46091318 0.6114701  0.6151172  0.54484683 0.60004145
 0.63365906 0.55278414 0.59576213 0.6487696  0.58636767 0.47622463
 0.6195855  0.4709046  0.58732903 0.4305575  0.52301335 0.50923723
 0.57919985 0.63262874 0.55146307 0.5498693  0.60074824 0.6320026
 0.6370218  0.6030973  0.5943333  0.62034535 0.56754833 0.5484992
 0.5458823  0.6076782  0.631516   0.6207136  0.59210306 0.59914637
 0

##Others

In [122]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/datadnn.csv', index_col=0, )
print(df[:6])

   year  da  NUM_COLLISIONS  temp  dewp     slp  visib  wdsp  mxpsd   gust  \
1  2020  24             524  37.3  33.7  1028.5    6.5   3.3    8.0  999.9   
2  2021  12             278  37.0  29.1  1019.0   10.0   6.5   12.0  999.9   
3  2021  22             254  36.5  28.4  1003.1   10.0   7.8   12.0   20.0   
4  2021  27             262  34.6  33.8  1012.8    8.0   7.8   12.0  999.9   
5  2021  26             263  31.9  23.4  1016.9    9.0   7.4   12.0  999.9   
6  2022  24             237  34.5  23.8  1010.6    9.7   7.4   12.0  999.9   

   ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1  ...    0    0    0    0    0    0    0    1    0    0  
2  ...    0    0    0    0    1    0    0    0    0    0  
3  ...    0    0    0    0    0    0    0    1    0    0  
4  ...    0    0    0    0    0    0    0    0    1    0  
5  ...    0    0    0    0    1    0    0    0    0    0  
6  ...    0    0    0    0    0    0    1    0    0    0  

[6 rows x 38 columns]


In [124]:
dnn = df.drop(columns=[ 'prcp', 'dewp','mxpsd','gust','slp','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
dnn = dnn.loc[dnn["year"] != 2012]
dnn = dnn.loc[dnn["year"] < 2020]
cols = dnn['NUM_COLLISIONS']
dnn = dnn.drop(columns=['NUM_COLLISIONS'])
dnn.insert(loc=26, column='NUM_COLLISIONS', value=cols)
print(dnn[:6])
dnn.describe()

    year  da  temp  visib  wdsp   max   min   sndp  Apr  Aug  ...  Nov  Oct  \
49  2016  28  35.0   10.0   4.3  46.0  23.0  999.9    0    0  ...    0    0   
51  2014  17  38.6    6.7   3.7  44.1  32.0  999.9    0    0  ...    0    0   
54  2016  25  33.5   10.0   6.5  37.9  30.0  999.9    0    0  ...    0    0   
55  2016  29  41.3   10.0   5.9  45.0  23.0  999.9    0    0  ...    0    0   
58  2017  20  39.9   10.0   4.3  45.0  37.0  999.9    0    0  ...    0    0   
59  2013  13  45.4    4.3   5.8  46.9  44.1  999.9    0    0  ...    0    0   

    Sep  Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
49    0    0    0    0    0    0    1             681  
51    0    0    0    0    1    0    0             589  
54    0    0    0    1    0    0    0             658  
55    0    0    0    0    1    0    0             645  
58    0    0    0    0    1    0    0             605  
59    0    0    1    0    0    0    0             373  

[6 rows x 27 columns]


,year,da,temp,visib,wdsp,max,min,sndp,Apr,Aug,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,2556.0,2556.000000,2556.000000,2556.000000,2556.000000,2556.000000,2556.000000,2.556000e+03,2556.000000,2556.000000,...,2556.000000,2556.000000,2556.000000,2556.000000,2556.000000,2556.000000,2556.000000,2556.000000,2556.000000,2556.000000
mean,2016.0,15.725743,51.487715,8.295618,10.682864,59.564280,43.869757,9.999000e+02,0.082160,0.084898,...,0.082160,0.084898,0.082160,0.143192,0.142801,0.142801,0.142801,0.142801,0.142801,599.118936
std,2.0,8.800168,14.162738,2.207870,4.242687,14.279867,14.722751,2.274182e-13,0.274661,0.278785,...,0.274661,0.278785,0.274661,0.350338,0.349939,0.349939,0.349939,0.349939,0.349939,100.258581
min,2013.0,1.000000,5.800000,0.200000,2.700000,18.000000,-2.000000,9.999000e+02,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2014.0,8.000000,40.400000,7.100000,7.700000,48.000000,33.100000,9.999000e+02,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000
50%,2016.0,16.000000,52.000000,9.400000,10.100000,60.100000,44.100000,9.999000e+02,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000
75%,2018.0,23.000000,63.900000,10.000000,13.000000,72.000000,55.900000,9.999000e+02,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,665.000000
max,2019.0,31.000000,77.500000,10.000000,39.300000,90.000000,71.600000,9.999000e+02,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [125]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = dnn.iloc[np.random.permutation(len(dnn))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

      year  da  temp  visib  wdsp   max   min   sndp  Apr  Aug  ...  May  Nov  \
684   2019  18  34.5   10.0  10.8  41.0  27.0  999.9    0    0  ...    0    0   
2200  2019  31  70.3    9.1   5.3  79.0  57.9  999.9    0    1  ...    0    0   
2075  2019   1  66.9   10.0  10.4  77.0  62.1  999.9    0    0  ...    0    0   
3588  2015  18  54.2    2.4  11.3  60.1  42.1  999.9    0    0  ...    0    0   
2101  2018  11  68.3    9.3   8.9  73.9  63.0  999.9    0    0  ...    0    0   
2730  2015  21  63.9   10.0  17.3  80.1  61.0  999.9    0    0  ...    0    0   

      Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
684     0    0    0    0    1    0    0    0  
2200    0    0    0    0    0    0    0    0  
2075    0    0    0    0    1    0    0    0  
3588    0    0    0    0    0    1    0    0  
2101    0    0    0    0    0    0    1    0  
2730    0    1    0    0    1    0    0    0  

[6 rows x 26 columns]


In [126]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

684     575
2200    502
2075    693
3588    767
2101    627
2730    624
Name: NUM_COLLISIONS, dtype: int64


In [127]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

# Define the number of output values (targets)
noutputs = 1
# Define the number of input values (predictors)
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)


26


In [128]:

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model', hidden_units=[17,13,9,7], optimizer=tf.train.AdamOptimizer(learning_rate=0.001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_COLLISIONS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff062d1d290>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model/model.ckpt.
INFO:tensorflow:loss = 62422.805, step = 1
INFO:tensorflow:global_step/sec: 460.124
INFO:tensorflow:loss = 0.9070146, step = 101 (0.220 sec)
INFO:tensorflow:global_step/sec: 657.54
INFO:tensorflow:loss = 0.21452484, step = 201 (0.152 sec)
INFO:tensorflow:global_step/sec: 683.969
INFO:tensorflow:loss = 0.20359117, step = 301 (0.146 sec)
INFO:tensorflow:global_step/sec: 656.558
INFO:tensorflow:loss = 0.17017865, step = 401 (0.155 sec)
INFO:tensorflow:global_step/sec: 633.407
INFO:tensorflow:loss = 0.17433774, step = 501 (0.156 sec)
INFO:tensorflow:global_step/sec: 576.177
INFO:tensorflow:loss = 0.17765565, step = 601 (0.173 sec)
INFO:tensorflow:global_step/sec: 635.605
INFO:tensorflow:loss = 0.18322639, step = 701 (0.157 sec)
INFO:tenso

DNNRegression has RMSE of 200.0920343269968
Just using average = 598.5009784735812 has RMSE of 102.40434664573948


In [129]:
print(predictors[trainsize:].values)
#print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model', hidden_units=[17,13,9,7], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff0633db250>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model', '_session_creation_timeout_secs': 7200}


[[2.018e+03 2.800e+01 4.180e+01 ... 1.000e+00 0.000e+00 0.000e+00]
 [2.019e+03 1.500e+01 4.770e+01 ... 0.000e+00 0.000e+00 0.000e+00]
 [2.019e+03 2.400e+01 4.650e+01 ... 0.000e+00 0.000e+00 1.000e+00]
 ...
 [2.017e+03 1.400e+01 6.710e+01 ... 0.000e+00 1.000e+00 0.000e+00]
 [2.019e+03 1.400e+01 3.900e+01 ... 0.000e+00 0.000e+00 1.000e+00]
 [2.018e+03 3.000e+01 3.190e+01 ... 0.000e+00 0.000e+00 0.000e+00]]


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.36530006 0.33201686 0.33876437 0.39548087 0.41206425 0.26414177
 0.38562542 0.25478423 0.4111431  0.30683085 0.43572986 0.55800706
 0.38391826 0.34203178 0.31752744 0.34235957 0.4200231  0.3278911
 0.43051374 0.29856765 0.3727539  0.42756575 0.27777952 0.45883965
 0.4030261  0.291053   0.35166708 0.35408887 0.40930805 0.29552352
 0.44592187 0.29199043 0.21379322 0.37832585 0.39681888 0.41883114
 0.14582789 0.452552   0.4109158  0.37340945 0.30726424 0.29460692
 0.23756367 0.35156873 0.3147342  0.31703484 0.256946   0.306409
 0.34769306 0.3773957  0.385972   0.41513005 0.48961103 0.4867388
 0.41107792 0.27690652 0.35358885 0.27408764 0.5044761  0.43070415
 0.44484153 0.45298895 0.40233502 0.24800393 0.37577367 0.3919853
 0.38498646 0.1967065  0.36263064 0.33329514 0.32758033 0.3548625
 0.13716313 0.33788556 0.4628078  0.32252157 0.2572303  0.38998067
 0.38326132 0.23711333 0.29624403 0.38244683 0.46537665 0.37485287
 0.30217564 0.43594217 0.33240566 0.31857488 0.28626344 0.32211396
 

## Location


REF https://www.geeksforgeeks.org/read-a-zipped-file-as-a-pandas-dataframe/ 12/11

In [130]:
df_loc = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/locdnn.zip', index_col=0,compression='zip' )
print(df_loc[:6])

   year  da  NUM_COLLISIONS   latitude  longitude  temp  dewp     slp  visib  \
1  2018   2               1  40.681750 -73.967480  14.7   2.0  1024.9   10.0   
2  2018   2               1  40.645370 -73.945110  14.7   2.0  1024.9   10.0   
3  2018   2               1  40.614830 -73.998380  14.7   2.0  1024.9   10.0   
4  2018   2               1  40.592190 -74.087395  14.7   2.0  1024.9   10.0   
5  2018   2               1  40.769817 -73.782370  14.7   2.0  1024.9   10.0   
6  2018   2               1  40.660175 -73.928200  14.7   2.0  1024.9   10.0   

   wdsp  ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1  12.9  ...    0    0    0    0    1    0    0    0    0    0  
2  12.9  ...    0    0    0    0    1    0    0    0    0    0  
3  12.9  ...    0    0    0    0    1    0    0    0    0    0  
4  12.9  ...    0    0    0    0    1    0    0    0    0    0  
5  12.9  ...    0    0    0    0    1    0    0    0    0    0  
6  12.9  ...    0    0    0    0    1    0    0  

In [131]:
df_loc_dnn = df_loc.drop(columns=[ 'prcp', 'dewp','mxpsd','gust','slp','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_loc_dnn = df_loc_dnn.loc[df_loc_dnn["year"] != 2012]
df_loc_dnn = df_loc_dnn.loc[df_loc_dnn["year"] < 2020]
cols = df_loc_dnn['NUM_COLLISIONS']
df_loc_dnn = df_loc_dnn.drop(columns=['NUM_COLLISIONS'])
df_loc_dnn.insert(loc=28, column='NUM_COLLISIONS', value=cols)
print(df_loc_dnn[:6])
df_loc_dnn.describe()

   year  da   latitude  longitude  temp  visib  wdsp   max  min   sndp  ...  \
1  2018   2  40.681750 -73.967480  14.7   10.0  12.9  18.0  5.0  999.9  ...   
2  2018   2  40.645370 -73.945110  14.7   10.0  12.9  18.0  5.0  999.9  ...   
3  2018   2  40.614830 -73.998380  14.7   10.0  12.9  18.0  5.0  999.9  ...   
4  2018   2  40.592190 -74.087395  14.7   10.0  12.9  18.0  5.0  999.9  ...   
5  2018   2  40.769817 -73.782370  14.7   10.0  12.9  18.0  5.0  999.9  ...   
6  2018   2  40.660175 -73.928200  14.7   10.0  12.9  18.0  5.0  999.9  ...   

   Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
1    0    0    0    1    0    0    0    0    0               1  
2    0    0    0    1    0    0    0    0    0               1  
3    0    0    0    1    0    0    0    0    0               1  
4    0    0    0    1    0    0    0    0    0               1  
5    0    0    0    1    0    0    0    0    0               1  
6    0    0    0    1    0    0    0    0    0          

,year,da,latitude,longitude,temp,visib,wdsp,max,min,sndp,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,...,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06
mean,2.016149e+03,1.566747e+01,4.072402e+01,-7.392063e+01,5.203610e+01,8.262622e+00,1.066076e+01,6.012695e+01,4.442143e+01,9.999000e+02,...,8.510758e-02,8.877971e-02,8.587448e-02,1.487626e-01,1.173738e-01,1.438814e-01,1.597195e-01,1.467698e-01,1.505395e-01,1.026582e+00
std,1.970562e+00,8.752606e+00,7.845179e-02,8.651997e-02,1.410404e+01,2.220453e+00,4.197681e+00,1.424273e+01,1.468128e+01,2.842172e-12,...,2.790418e-01,2.844256e-01,2.801787e-01,3.558544e-01,3.218653e-01,3.509695e-01,3.663458e-01,3.538765e-01,3.575996e-01,1.784039e-01
min,2.013000e+03,1.000000e+00,4.049895e+01,-7.425453e+01,5.800000e+00,2.000000e-01,2.700000e+00,1.800000e+01,-2.000000e+00,9.999000e+02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,2.014000e+03,8.000000e+00,4.066895e+01,-7.397644e+01,4.100000e+01,7.000000e+00,7.700000e+00,4.890000e+01,3.310000e+01,9.999000e+02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
50%,2.016000e+03,1.600000e+01,4.072234e+01,-7.392891e+01,5.300000e+01,9.300000e+00,1.010000e+01,6.100000e+01,4.500000e+01,9.999000e+02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
75%,2.018000e+03,2.300000e+01,4.076838e+01,-7.386641e+01,6.430000e+01,1.000000e+01,1.290000e+01,7.300000e+01,5.700000e+01,9.999000e+02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
max,2.019000e+03,3.100000e+01,4.091288e+01,-7.366301e+01,7.750000e+01,1.000000e+01,3.930000e+01,9.000000e+01,7.160000e+01,9.999000e+02,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.100000e+01


In [133]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_loc_dnn.iloc[np.random.permutation(len(df_loc_dnn))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

         year  da   latitude  longitude  temp  visib  wdsp   max   min   sndp  \
1144965  2019  16  40.667835  -73.89484  41.1    7.9  10.6  48.9  30.0  999.9   
780661   2014  27  40.836869  -73.94894  63.9    9.8  13.6  75.0  59.0  999.9   
123122   2017   9  40.666150  -73.80575  45.9    9.1  17.7  52.0  41.0  999.9   
1530982  2018   6  40.650490  -74.01179  55.5   10.0   6.0  66.9  46.0  999.9   
1441299  2019   6  40.754770  -73.82997  68.9    1.5   7.5  73.9  59.0  999.9   
699748   2016  27  40.660860  -73.89883  40.0    9.8   5.4  46.9  37.0  999.9   

         ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1144965  ...    0    0    0    0    0    0    0    0    0    0  
780661   ...    0    0    0    0    0    0    0    1    0    0  
123122   ...    0    0    0    0    0    0    0    0    0    1  
1530982  ...    0    0    1    0    0    0    0    0    0    0  
1441299  ...    0    0    0    0    0    0    0    0    0    0  
699748   ...    0    0    0    0    0    1

In [134]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

1144965    2
780661     1
123122     1
1530982    1
1441299    1
699748     1
Name: NUM_COLLISIONS, dtype: int64


In [135]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

# Define the number of output values (targets)
noutputs = 1
# Define the number of input values (predictors)
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)


28


In [154]:

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model_loc', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_loc', hidden_units=[16,7,3], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_COLLISIONS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff05e586910>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_loc', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model_loc/model.ckpt.
INFO:tensorflow:loss = 88380.15, step = 1
INFO:tensorflow:global_step/sec: 734.647
INFO:tensorflow:loss = 0.010075731, step = 101 (0.141 sec)
INFO:tensorflow:global_step/sec: 911.274
INFO:tensorflow:loss = 0.009417284, step = 201 (0.110 sec)
INFO:tensorflow:global_step/sec: 916.185
INFO:tensorflow:loss = 0.0086057875, step = 301 (0.109 sec)
INFO:tensorflow:global_step/sec: 1033.76
INFO:tensorflow:loss = 0.007711338, step = 401 (0.096 sec)
INFO:tensorflow:global_step/sec: 941.208
INFO:tensorflow:loss = 0.0067865015, step = 501 (0.108 sec)
INFO:tensorflow:global_step/sec: 906.806
INFO:tensorflow:loss = 0.005872124, step = 601 (0.108 sec)
INFO:tensorflow:global_step/sec: 946.2
INFO:tensorflow:loss = 0.004995689, step = 701 (0.105 sec

DNNRegression has RMSE of 0.18163767653611632
Just using average = 1.0265051171123096 has RMSE of 0.1815919467771684


In [155]:
print(predictors[trainsize:].values)
#print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_loc', hidden_units=[16,7,3], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff0631b53d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_loc', '_session_creation_timeout_secs': 7200}


[[2.01800000e+03 2.90000000e+01 4.07349930e+01 ... 0.00000000e+00
  1.00000000e+00 0.00000000e+00]
 [2.01300000e+03 1.50000000e+01 4.07645465e+01 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.01500000e+03 2.20000000e+01 4.08282960e+01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [2.01400000e+03 1.00000000e+00 4.05958903e+01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.01500000e+03 1.00000000e+00 4.09046015e+01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.01600000e+03 1.10000000e+01 4.08534620e+01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_loc/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.00088096 0.00088096 0.00088096 ... 0.00088096 0.00088096 0.00088096]
[0.00086133 0.00086133 0.00086133 ... 0.00086133 0.00086133 0.00086133]
The trained model has an aproximate error rate of 0.004093819009957885 which equates to 0%
